In [1]:
from selenium import webdriver

#### 파일 업로드
- google vision api 페이지 활용
- vision api 페이지로 이동
- iframe으로 frame을 변경
- 이미지 파일을 업로드
- 웹데이터와 문자열 데이터(document) 수집

In [3]:
# google vision api 페이지 이동
url = "https://cloud.google.com/vision/"
driver = webdriver.Chrome()
driver.get(url)

In [5]:
# main frame --> iframe으로 영역 변경
iframe = driver.find_element_by_css_selector("#vision_demo_section > iframe")
driver.switch_to_frame(iframe)

In [6]:
# 다시 메인 frame으로 돌아오기 : driver.switch_to_default_content()

In [7]:
# 파일 업로드(절대경로)
# 관리자도구에서 "input file"키워드 찾기
file_path = "C:/dss/dev/TIL/web_programming/dog.png"
driver.find_element_by_css_selector("#input").send_keys(file_path)

In [8]:
# web버튼 클릭하자
driver.find_element_by_css_selector("#webDetection").click()

In [21]:
results = driver.find_elements_by_css_selector("#card > .container > .row")[:15]

for result in results:
    print(
    result.find_element_by_css_selector('.name').text,
    result.find_element_by_css_selector('.score').text
    )

French Bulldog 1.29119
Bulldog 1.27994
Boston Terrier 1.27595
Soft-coated Wheaten Terrier 1.04689
Puppy 0.98325
Bull Terrier 0.9654
Border Collie 0.82782
Terrier 0.6867
Pet 0.6386
Coat 0.5719
Brindle 0.5711
Dog breed 0.5109
Breed 0.4445
Black Brindle 0.4197
Dog 0


In [22]:
# document 클릭하여 문자열 데이터 가져오기
driver.find_element_by_css_selector("#fullTextAnnotation").click()

In [23]:
fieldsets = driver.find_elements_by_css_selector(".page > .container > .block")
len(fieldsets)

2

In [38]:
documents = []

for fieldset in fieldsets:
    words = fieldset.find_elements_by_css_selector(".paragraph > .container > .word")
#     print(len(words))
    word_list = []
    for word in words:
#         print(word.text)
        word = word.text.replace(" ","").replace("'","")
#         print(word)
        word_list.append(word)
    documents.append(" ".join(word_list))
documents

['In the dog house', ' 인기를 잃다 , 면목을 잃다']

In [39]:
driver.quit()

##### gmarket 베스트 200 데이터 수집

In [2]:
url = "http://corners.gmarket.co.kr/Bestsellers"
driver = webdriver.Chrome()
driver.get(url)

In [10]:
items = driver.find_elements_by_css_selector(".best-list:nth-child(3) > ul >  li ")
len(items)

200

In [20]:
best_list = []
for item in items:
    best_list.append({
        'link' : item.find_element_by_css_selector(".itemname").get_attribute("href"),
        'title' : item.find_element_by_css_selector(".itemname").text,
        'o_price' : item.find_element_by_css_selector(".item_price > .o-price").text.replace(",","")[:-1],
        's_price' : item.find_element_by_css_selector(".item_price > .s-price > strong > span").text.replace(",","")[:-1],
    }) 


In [26]:
df = pd.DataFrame(best_list)
df.head()

,link,o_price,s_price,title
0,http://item.gmarket.co.kr/Item?goodscode=67170...,30000,9900,르까프 케이스위스 7부팬츠 반바지 티셔츠 가격다운
1,http://item.gmarket.co.kr/Item?goodscode=97974...,80000,16900,[벨리에르]소프트플라넬 워싱 스프레드 1+1 (퀸)
2,http://item.gmarket.co.kr/Item?goodscode=12188...,24900,14900,[웅진]빅토리아 탄산수 탄산음료 500mlx40 350mlx48캔 택2
3,http://item.gmarket.co.kr/Item?goodscode=22627...,38900,34900,2018년 에어솔솔 썸머팬티 / 밴드 2팩 +신세계상품권
4,http://item.gmarket.co.kr/Item?goodscode=12854...,21600,6500,국산 면스판 나시/속바지 3매


In [30]:
idx = df['o_price'] == ""
df['o_price'][idx] = df['s_price'][idx]
df['o_price'][idx]

28      5000
34     19400
46     96000
60     11800
68      5900
83     19800
87     79000
107     7900
117    12500
118     5900
130    23500
149     7000
157     6900
Name: o_price, dtype: object

In [48]:
df['discount'] = \
(1 - round((df['s_price'].astype('int') / df['o_price'].astype('int')),2))* 100 

In [51]:
df['discount'] = df['discount'].astype('int')
df['discount'] = df['discount'].astype('str') + "%"
df['discount']

0      67%
1      79%
2      40%
3       9%
4      70%
5      70%
6      70%
7      63%
8      52%
9      63%
10     70%
11     70%
12     69%
13     70%
14     35%
15     69%
16     69%
17     70%
18     17%
19      9%
20      9%
21     70%
22     70%
23     43%
24     69%
25     70%
26     70%
27     50%
28      0%
29     25%
      ... 
170    70%
171    70%
172    70%
173    12%
174    70%
175    18%
176    19%
177    35%
178    70%
179     6%
180    59%
181    70%
182    70%
183    70%
184    14%
185    21%
186    70%
187     7%
188     2%
189    70%
190    70%
191    70%
192    70%
193    70%
194     6%
195    54%
196    70%
197     5%
198    55%
199    15%
Name: discount, Length: 200, dtype: object

In [52]:
df.tail()

,link,o_price,s_price,title,discount
195,http://item.gmarket.co.kr/Item?goodscode=62686...,66000,30200,(공식판매처/QR발송) 캐리비안 베이 종일권/자켓/방수팩패키지,54%
196,http://item.gmarket.co.kr/Item?goodscode=25604...,33000,9900,힐슈빌 여성샌들 여름샌들 샌들 스트랩 웨지 플랫샌들,70%
197,http://item.gmarket.co.kr/Item?goodscode=14020...,168000,159000,[휴스톰]듀얼라이팅 무선 물걸레청소기 HS-9000 TV CF동일상품,5%
198,http://item.gmarket.co.kr/Item?goodscode=87419...,39900,17900,닥터방기원 탈모샴푸 탈모증상완화 임상시험완료,55%
199,http://item.gmarket.co.kr/Item?goodscode=15138...,48000,40900,[맥심]쿠폰가36900원~/화이트/모카골드 420T/400T/커피믹스,15%


In [53]:
driver.quit()